# 2.2 영화 평점 데이터
---

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

## users, ratings, movies

In [15]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_csv('users.dat', delimiter='::', header=None, names=unames)

# rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
# ratings = pd.read_csv('ratings.dat', delimiter='::', header=None, names=rnames)
# pd.to_pickle(ratings,'ratings.pkl')
ratings = pd.read_pickle('ratings.pkl')

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=mnames)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  # Remove the CWD from sys.path while we load stuff.


In [5]:
display(users.info()) # users.describe()
display(ratings.info())
display(movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
user_id       6040 non-null int64
gender        6040 non-null object
age           6040 non-null int64
occupation    6040 non-null int64
zip           6040 non-null object
dtypes: int64(3), object(2)
memory usage: 236.0+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
user_id      1000209 non-null int64
movie_id     1000209 non-null int64
rating       1000209 non-null int64
timestamp    1000209 non-null int64
dtypes: int64(4)
memory usage: 30.5 MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
movie_id    3883 non-null int64
title       3883 non-null object
genres      3883 non-null object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


None

In [6]:
display(users.head(), ratings.head(), movies.head())

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## data
- 세 개의 테이블을 합친 테이블

In [7]:
data=pd.merge(pd.merge(ratings,users),movies)
data[:5]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


## gender_rating
- 영화별 남성/여성 점수 평균

In [8]:
gender_rating = data.pivot_table('rating',index='title',columns='gender',aggfunc='mean')
gender_rating[:5]

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024


## mean_ratings
- 영화별 남성/여성 평균에서, 250개 이상의 평가를 받은 영화만 선택

In [16]:
df = pd.merge(gender_rating, data.groupby('title').size().to_frame('r_count'), left_index=True, right_index=True)
mean_ratings = df[df.r_count>250]
mean_ratings.head()

,F,M,r_count
title,,,
"'burbs, The (1989)",2.793478,2.962085,303
10 Things I Hate About You (1999),3.646552,3.311966,700
101 Dalmatians (1961),3.791444,3.500000,565
101 Dalmatians (1996),3.240000,2.911215,364
12 Angry Men (1957),4.184397,4.328421,616
"13th Warrior, The (1999)",3.112000,3.168000,750
2 Days in the Valley (1996),3.488889,3.244813,286
"20,000 Leagues Under the Sea (1954)",3.670103,3.709205,575
2001: A Space Odyssey (1968),3.825581,4.129738,1716


In [10]:
ratings_by_title = data.groupby('title').size()
active_titles = ratings_by_title[ratings_by_title >= 250].index

mean_ratings = gender_rating.loc[active_titles]
mean_ratings.head()

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421


In [11]:
display(mean_ratings.sort_values('F',ascending=False)[:10])
display(mean_ratings.sort_values('M',ascending=False)[:10])

gender,F,M
title,,
"Close Shave, A (1995)",4.644444,4.473795
"Wrong Trousers, The (1993)",4.588235,4.478261
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Schindler's List (1993),4.562602,4.491415
"Shawshank Redemption, The (1994)",4.539075,4.560625
"Grand Day Out, A (1992)",4.537879,4.293255
To Kill a Mockingbird (1962),4.536667,4.372611
Creature Comforts (1990),4.513889,4.272277


gender,F,M
title,,
"Godfather, The (1972)",4.314700,4.583333
Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954),4.481132,4.576628
"Shawshank Redemption, The (1994)",4.539075,4.560625
Raiders of the Lost Ark (1981),4.332168,4.520597
"Usual Suspects, The (1995)",4.513317,4.518248
Star Wars: Episode IV - A New Hope (1977),4.302937,4.495307
Schindler's List (1993),4.562602,4.491415
"Wrong Trousers, The (1993)",4.588235,4.478261
"Close Shave, A (1995)",4.644444,4.473795


In [12]:
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']
mean_ratings.sort_values('diff',ascending=False)[:10]

gender,F,M,diff
title,,,
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
Dumb & Dumber (1994),2.697987,3.336595,0.638608
"Longest Day, The (1962)",3.411765,4.031447,0.619682
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
Evil Dead II (Dead By Dawn) (1987),3.297297,3.909283,0.611985
"Hidden, The (1987)",3.137931,3.745098,0.607167
Rocky III (1982),2.361702,2.943503,0.581801
Caddyshack (1980),3.396135,3.969737,0.573602


In [13]:
std_by_title = data.groupby('title')['rating'].std().loc[active_titles]
std_by_title.sort_values(ascending=False)[:10]

title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
Eyes Wide Shut (1999)                    1.259624
Evita (1996)                             1.253631
Billy Madison (1995)                     1.249970
Fear and Loathing in Las Vegas (1998)    1.246408
Bicentennial Man (1999)                  1.245533
Name: rating, dtype: float64

In [14]:
tbl = pd.merge(mean_ratings, std_by_title.to_frame('std'), left_index=True, right_index=True)
tbl.sort_values('std',ascending=False)[:15]

,F,M,diff,std
title,,,,
Dumb & Dumber (1994),2.697987,3.336595,0.638608,1.321333
"Blair Witch Project, The (1999)",3.038732,3.029381,-0.009351,1.316368
Natural Born Killers (1994),3.192982,3.134812,-0.058170,1.307198
Tank Girl (1995),2.901408,2.543554,-0.357854,1.277695
"Rocky Horror Picture Show, The (1975)",3.673016,3.160131,-0.512885,1.260177
Eyes Wide Shut (1999),2.891213,3.305949,0.414736,1.259624
Evita (1996),3.460177,3.118343,-0.341834,1.253631
Billy Madison (1995),3.141026,3.194946,0.053920,1.249970
Fear and Loathing in Las Vegas (1998),3.079365,3.417021,0.337656,1.246408
